Julien Gauthier

# Reinforcement Learning (Deep Q) Project

#### WARNING : install the correct versions of gym/tensorflow/keras-rl2 in a virtual environment.

In [8]:
# These are the correct versions for this project to work :
!pip install tensorflow==2.12.0 keras-rl2==1.0.5 gym==0.25.2

### I. Setting up the OpenAI Cart Pole environment

In [1]:
import gym
import random

The environment is where the experiment takes place, the states are the different input parameters (in this case : cart position, cart velocity, pole angle, pole tip velocity) and the actions are the output possibilities (move the cart left or right).

In [2]:
env = gym.make("CartPole-v1", render_mode="human")
states = env.observation_space.shape[0]
actions = env.action_space.n

c:\Users\julie\OneDrive\Bureau\ENSEEIHT\projet-perso\reinforcement-learning-cart\.venv\Lib\site-packages\gym\core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\julie\OneDrive\Bureau\ENSEEIHT\projet-perso\reinforcement-learning-cart\.venv\Lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


Testing the environment with random actions.

In [24]:
episodes = 10
for episode in range(1, episodes + 1) :
    state = env.reset()
    score = 0
    done = False
    
    while not done :
        env.render()
        action = random.choice([0, 1])
        next_state, reward, done, info = env.step(action)[:-1] # The last element is useless.
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

c:\Users\julie\OneDrive\Bureau\ENSEEIHT\projet-perso\reinforcement-learning-cart\.venv\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode:1 Score:13.0
Episode:2 Score:12.0
Episode:3 Score:23.0
Episode:4 Score:23.0
Episode:5 Score:19.0
Episode:6 Score:18.0
Episode:7 Score:11.0
Episode:8 Score:10.0
Episode:9 Score:14.0
Episode:10 Score:14.0


### II. Deep Learning model with Keras.

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam

c:\Users\julie\OneDrive\Bureau\ENSEEIHT\projet-perso\reinforcement-learning-cart\.venv\Lib\site-packages\tensorflow\python\debug\cli\debugger_cli_common.py:19: DeprecationWarning: module 'sre_constants' is deprecated
  import sre_constants


Let's create a function that will build our model.

In [4]:
def build_model(states, actions) :
    model = Sequential()
    model.add(Flatten(input_shape=(1, states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

We can now use our function to create and show an instance of the model :

In [5]:
model = build_model(states, actions)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 4)                 0         
                                                                 
 dense (Dense)               (None, 24)                120       
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


### III. Agent creation with Keras-RL

In [7]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory